## Part 2 : Implement a chatot using chatbot corpus for Olympus customer support 

###### We will implement semi-rule based ,machine learning based chatbot . We will split the corpus into intents/patterns, tags and responses. Intents/patterns will act  our independent variables whereas tags will be our dependent variable. We will preprocess, tokenize and vectorize our patterns using Bag of words and keras tokenizer and attempt to predict the tags. Based on tags,random responses will be picked and produced to the user.

In [1]:
#Import libraries
import nltk
from nltk.chat.util import Chat,reflections
from google.colab import  drive
import os
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
from nltk import word_tokenize
import tensorflow
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout,Input,Embedding,Flatten,BatchNormalization,GlobalAveragePooling1D
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.initializers import GlorotNormal,GlorotUniform,HeNormal,HeUniform
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers import SGD
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [134]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [135]:
path ='/content/drive/MyDrive/Great Learning/NLP/'

#### 1. Import the chatbot corpus and split it into patterns ,tags and responses.

In [136]:
#Read chatbot corpus
ChatBot_corpus= pd.read_json(path+'GL Bot.json')

In [137]:
type(ChatBot_corpus)

pandas.core.frame.DataFrame

In [138]:
ChatBot_corpus

,intents
0,"{'tag': 'Intro', 'patterns': ['hi', 'how are y..."
1,"{'tag': 'Exit', 'patterns': ['thank you', 'tha..."
2,"{'tag': 'Olympus', 'patterns': ['olympus', 'ex..."
3,"{'tag': 'SL', 'patterns': ['i am not able to u..."
4,"{'tag': 'NN', 'patterns': ['what is deep learn..."
5,"{'tag': 'Bot', 'patterns': ['what is your name..."
6,"{'tag': 'timings', 'patterns': ['when are your..."
7,"{'tag': 'Profane', 'patterns': ['what the hell..."
8,"{'tag': 'Ticket', 'patterns': ['my problem is ..."


In [139]:
#Separate patterns/intents, tags and responses
Words=[]
X_feat=[]
tags=[]
response= []
labels=[]
tags_ext=[]
for intent in ChatBot_corpus['intents']:
  #print(intent)
  for pattern in intent['patterns']:
    #print(pattern)
   
    words = nltk.sent_tokenize(pattern)
    
    #print(words)
    Words.extend(words)
    X_feat.append(words)
    tags.append(intent['tag'])
    #print("words {}, tags : {}".format(words,intent['tag']))
    response.append(intent['responses'])
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

In [140]:
len(Words)

259

In [141]:
len(tags)

259

#### 2. Preprocess and vectorize patterns. One hot encode the tag column.

In [142]:
#OneHotEncode the tags
ohe = OneHotEncoder()
tags=np.array(tags).reshape(-1,1)
tags_ohe = ohe.fit_transform(tags)

In [143]:
#Clean the patterns/intents text
wnLemm = WordNetLemmatizer()
import re
#def cleanData(X):
for i in range(len(Words)):
  cleanTxt=Words[i]
  cleanTxt=re.sub('[\'\"\.\,(){}]'," ",cleanTxt)
  cleanTxt=re.sub('[!@\*?:;#\$\%^&~]'," ",cleanTxt)
  cleanTxt=re.sub('[+-\/X*]'," ",cleanTxt)
  cleanTxt=re.sub('[\[\]=<>]'," ",cleanTxt)
  cleanTxt=re.sub('[<<>>_*]'," ",cleanTxt)
  cleanTxt=re.sub('[^a-zA-Z]'," ",cleanTxt)
  cleanTxt=cleanTxt.lower()
  words_token = cleanTxt.split()
  w = [wnLemm.lemmatize(word) for word in words_token] 
  w = " ".join(w)
  Words[i]=w


In [145]:
#Vectoization of split intents using Bag of words
bow=CountVectorizer(ngram_range=(1, 3), max_features=10000)
rnd_os=SMOTE(k_neighbors=6)
def implementCountVectorizer(Words):
  
  features_bow= bow.fit_transform(Words).todense()
  X_feat_os,tags_ohe_os= rnd_os.fit_resample(features_bow,tags_ohe.todense())
  X_train,X_test,y_train,y_test=train_test_split(X_feat_os,tags_ohe_os,random_state=42,test_size=0.30)
  return X_train,X_test,y_train,y_test

In [14]:
#Vectorization using tokeniser
vocab_size = 5000
embedding_dim = 3000
max_len = 20
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_token)

In [15]:
def implementTokenizer(Words):
  tokenizer.fit_on_texts(Words)
  sequences = tokenizer.texts_to_sequences(Words)
  X_feat_seq = pad_sequences(sequences, truncating='pre', maxlen=max_len)
  return X_feat_seq

In [146]:
#Split bag of words into train, test
X_train_bow,X_test_bow,y_train_bow,y_test_bow=implementCountVectorizer(Words)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [147]:
X_feat_seq=implementTokenizer(Words)

In [148]:
Words[10:15]

['please help me',
 'i am learner from',
 'i belong to',
 'aiml batch',
 'aifl batch']

In [149]:
X_feat_seq[10:15]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 126,  58,  50],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,  13,  20, 288,  40],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  13, 289,   4],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 290, 127],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 291, 127]], dtype=int32)

In [167]:
#Split tokenizer into train,test
X_train,X_test,y_train,y_test=train_test_split(X_feat_seq,tags_ohe.toarray())

#### 3. Create and tune the model.

##### 3.1. Create ANN model

In [168]:
#Form embedding matrix using glove embedding to be used in neural network
embeddings_index = dict()
f = open('/content/drive/MyDrive/Great Learning/NLP/glove.6B.50d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
vocab_size = len(tokenizer.word_index) + 1

Loaded 400001 word vectors.


In [169]:
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  #print(len(embedding_vector))
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [249]:
#Define ANN
def train_test_model(Lambda,lr, epochs):
  wt_initializer=HeNormal()
  model = Sequential()
  model.add(Input(shape=(max_len) ))
  model.add(Embedding(vocab_size, 50,weights=[embedding_matrix]))
  model.add(GlobalAveragePooling1D())
  model.add(Flatten())
  #model.add(Dropout(0.2))
  model.add(BatchNormalization())
  model.add(Dense(16, activation='relu',kernel_initializer=wt_initializer))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())
  model.add(Dense(9, activation='sigmoid',kernel_initializer=wt_initializer,kernel_regularizer=tensorflow.keras.regularizers.l2(Lambda)))
  tsgd = SGD(learning_rate=lr)
  # compile the model
  model.compile(optimizer=tsgd, loss='binary_crossentropy', metrics=['accuracy'])
  model.fit(X_train,y_train, epochs=epochs, batch_size=16, verbose=1,validation_split=0.20)
  acc=model.evaluate(X_test,y_test)
  return acc,model
# summarize the model


In [208]:
#Tune ANN
import math
for i in range (1,6):
    lr = math.pow(10, np.random.uniform(-4.0, 0))
    Lambda = math.pow(10, np.random.uniform(-5,-3))
    acc,model_sgd = train_test_model( Lambda,lr, 20)
    print("try {0}/{1} Best_val_acc: {2}, lr: {3}, Lambda: {4}\n ".format(i,5 ,acc, lr, Lambda))



Epoch 1/20
5/5 [==============================] - 1s 46ms/step - loss: 0.7010 - accuracy: 0.1448 - val_loss: 0.7052 - val_accuracy: 0.2449
Epoch 2/20
5/5 [==============================] - 0s 6ms/step - loss: 0.6911 - accuracy: 0.1793 - val_loss: 0.7046 - val_accuracy: 0.2449
Epoch 3/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7040 - accuracy: 0.0966 - val_loss: 0.7039 - val_accuracy: 0.2449
Epoch 4/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7062 - accuracy: 0.1655 - val_loss: 0.7032 - val_accuracy: 0.2449
Epoch 5/20
5/5 [==============================] - 0s 6ms/step - loss: 0.6990 - accuracy: 0.1862 - val_loss: 0.7026 - val_accuracy: 0.2449
Epoch 6/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7066 - accuracy: 0.1310 - val_loss: 0.7020 - val_accuracy: 0.2449
Epoch 7/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7000 - accuracy: 0.1379 - val_loss: 0.7013 - val_accuracy: 0.2449
Epoch 8/20
5/5 [=================

In [26]:
model_sgd.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 50)            28800     
_________________________________________________________________
global_average_pooling1d_4 ( (None, 50)                0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 50)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 50)                200       
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 459       
Total params: 29,459
Trainable params: 29,359
Non-trainable params: 100
_________________________________________________________________


In [180]:
print("Trainings score :",model_sgd.evaluate(X_train, y_train))
print("Testing score :",model_sgd.evaluate(X_test,y_test))

7/7 [==============================] - 0s 2ms/step - loss: 0.3407 - accuracy: 0.4124
Trainings score : [0.3407194912433624, 0.41237112879753113]
3/3 [==============================] - 0s 4ms/step - loss: 0.3400 - accuracy: 0.3692
Testing score : [0.3400239944458008, 0.3692307770252228]


In [251]:
scc,model_sgd=train_test_model(0.0003334509699429611 ,0.415988399710898,100)

Epoch 1/100
10/10 [==============================] - 1s 20ms/step - loss: 0.7333 - accuracy: 0.1806 - val_loss: 0.5463 - val_accuracy: 0.0513
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5136 - accuracy: 0.2387 - val_loss: 0.4114 - val_accuracy: 0.2051
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3941 - accuracy: 0.2968 - val_loss: 0.3435 - val_accuracy: 0.3333
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3327 - accuracy: 0.3355 - val_loss: 0.3135 - val_accuracy: 0.4103
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3205 - accuracy: 0.3742 - val_loss: 0.2963 - val_accuracy: 0.4359
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3020 - accuracy: 0.3935 - val_loss: 0.2879 - val_accuracy: 0.4359
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.2958 - accuracy: 0.3806 - val_loss: 0.2820 - val_accuracy: 0.5128
Epoch 8/100


In [252]:
print("Trainings score :",model_sgd.evaluate(X_train, y_train))
print("Testing score :",model_sgd.evaluate(X_test,y_test))

7/7 [==============================] - 0s 2ms/step - loss: 0.1111 - accuracy: 0.8918
Trainings score : [0.1110503077507019, 0.8917526006698608]
3/3 [==============================] - 0s 3ms/step - loss: 0.1839 - accuracy: 0.7385
Testing score : [0.18387246131896973, 0.7384615540504456]


In [253]:
predict=model_sgd.predict(X_test)

In [254]:
y_test_inv=ohe.inverse_transform(y_test)
pred_inv= ohe.inverse_transform(predict)

In [255]:
#predict using ANN for 10 records from dataset
for ind in range(10,21):
  print("\nActual value : ",y_test_inv[ind])
  print("Predicted value :",pred_inv[ind])


Actual value :  ['SL']
Predicted value : ['NN']

Actual value :  ['NN']
Predicted value : ['NN']

Actual value :  ['Bot']
Predicted value : ['Exit']

Actual value :  ['SL']
Predicted value : ['SL']

Actual value :  ['timings']
Predicted value : ['timings']

Actual value :  ['Exit']
Predicted value : ['Exit']

Actual value :  ['NN']
Predicted value : ['SL']

Actual value :  ['SL']
Predicted value : ['SL']

Actual value :  ['Ticket']
Predicted value : ['Intro']

Actual value :  ['NN']
Predicted value : ['NN']

Actual value :  ['SL']
Predicted value : ['SL']


##### 3.2 Create and tune machine learning models

In [273]:
logRegression=LogisticRegression(C=1.0,penalty='l1',solver='liblinear',max_iter =100,multi_class='ovr')
multiNomialNB =MultinomialNB()

svm=SVC(C=1.0,kernel='rbf')

In [270]:
#Train and Tune Machine learning algorithms
param_grid_rf ={'n_estimators':list(range(1,300)),'criterion':['gini','entropy'],'max_depth':list(range(1,100))}
param_grid_decisionCl = {'criterion':['gini','entropy'],
                         'splitter':['best','random'],
                         'max_depth':list(range(1,200)),
                         'max_features':['auto','log2'],
                         'max_leaf_nodes':list(range(1,200))
                        }


In [271]:
rs_rf = RandomizedSearchCV(RandomForestClassifier(),param_grid_rf,n_iter=10,cv=5,verbose=1)
rs_dcl = RandomizedSearchCV(DecisionTreeClassifier(),param_grid_decisionCl,n_iter=10,cv=5,verbose=1)


In [275]:
#predict using BoW
model_dict = {'logRegression: ':logRegression,
              'MultinomialNB': multiNomialNB,
              'svm':svm,
              'RandomForest':rs_rf,
              'decisionTree':rs_dcl
              }

In [278]:
for key,value in model_dict.items():
  model=OneVsRestClassifier(value)
  model.fit(X_train_bow,y_train_bow)
  predict = model.predict(X_test_bow)
  print("=======Model : {}=======".format(key))
  print("Accuracy : ",accuracy_score(y_test_bow,predict))
  print("classification_report : \n",classification_report(y_test_bow,predict))


=======Model : logRegression: =======
Accuracy :  0.32323232323232326
classification_report : 
               precision    recall  f1-score   support

           0       0.92      0.48      0.63        23
           1       1.00      0.13      0.23        31
           2       1.00      0.12      0.22        16
           3       1.00      0.50      0.67        16
           4       1.00      0.90      0.95        20
           5       1.00      0.32      0.48        19
           6       0.89      0.29      0.43        28
           7       1.00      0.09      0.16        23
           8       0.86      0.27      0.41        22

   micro avg       0.96      0.33      0.49       198
   macro avg       0.96      0.34      0.46       198
weighted avg       0.96      0.33      0.45       198
 samples avg       0.33      0.33      0.33       198

=======Model : MultinomialNB=======
Accuracy :  0.3333333333333333
classification_report : 
               precision    recall  f1-score   suppor

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


=======Model : svm=======
Accuracy :  0.21212121212121213
classification_report : 
               precision    recall  f1-score   support

           0       1.00      0.30      0.47        23
           1       0.00      0.00      0.00        31
           2       1.00      0.12      0.22        16
           3       1.00      0.06      0.12        16
           4       1.00      0.75      0.86        20
           5       1.00      0.21      0.35        19
           6       0.86      0.21      0.34        28
           7       1.00      0.09      0.16        23
           8       1.00      0.23      0.37        22

   micro avg       0.98      0.21      0.35       198
   macro avg       0.87      0.22      0.32       198
weighted avg       0.82      0.21      0.31       198
 samples avg       0.21      0.21      0.21       198

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   19.4s finished


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
=======Model : decisionTree=======
Accuracy :  0.3484848484848485
classification_report : 
               precision    recall  f1-score   support

           0       0.58      0.61      0.60        23
           1       0.88      0.23      0.36        31
           2       0.86      0.38      0.52        16
           3       0.71      0.31      0.43        16
           4       0.74      0.85      0.79        20
           5       0.58      0.37      0.45        19
           6       0.93      0.50      0.65        28
           7       0.67      0.09      0.15        23
           8       0.83      0.23      0.36        22

   micro avg       0.73      0.39      0.51       198
   macro avg       0.75      0.39      0.48       198
weighted avg       0.76      0.39      0.48       198
 samples avg       0.37      0.39      0.37       198



[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [279]:
rs_rf.fit(X_train_bow,y_train_bow)
predict_rs_rf = rs_rf.predict(X_test_bow)
print("Accuracy : ",accuracy_score(y_test_bow,predict_rs_rf))
print("classification_report : \n",classification_report(y_test_bow,predict_rs_rf))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   34.4s finished


Accuracy :  0.37373737373737376
classification_report : 
               precision    recall  f1-score   support

           0       0.94      0.65      0.77        23
           1       1.00      0.32      0.49        31
           2       1.00      0.38      0.55        16
           3       0.80      0.25      0.38        16
           4       1.00      0.80      0.89        20
           5       1.00      0.32      0.48        19
           6       0.90      0.32      0.47        28
           7       1.00      0.09      0.16        23
           8       1.00      0.27      0.43        22

   micro avg       0.96      0.37      0.54       198
   macro avg       0.96      0.38      0.51       198
weighted avg       0.96      0.37      0.51       198
 samples avg       0.37      0.37      0.37       198



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [281]:
rs_dcl.fit(X_train_bow,y_train_bow)
predict_rs_dcl = rs_dcl.predict(X_test_bow)
print("Accuracy : ",accuracy_score(y_test_bow,predict_rs_dcl))
print("classification_report : \n",classification_report(y_test_bow,predict_rs_dcl))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy :  0.43434343434343436
classification_report : 
               precision    recall  f1-score   support

           0       0.79      0.65      0.71        23
           1       0.81      0.42      0.55        31
           2       0.70      0.44      0.54        16
           3       0.56      0.31      0.40        16
           4       0.89      0.80      0.84        20
           5       0.86      0.32      0.46        19
           6       0.84      0.57      0.68        28
           7       1.00      0.09      0.16        23
           8       0.86      0.27      0.41        22

   micro avg       0.80      0.43      0.56       198
   macro avg       0.81      0.43      0.53       198
weighted avg       0.82      0.43      0.54       198
 samples avg       0.43      0.43      0.43       198



[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.6s finished
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Comparing the accuracies of models, we observe that an ANN performs the best. So we will use ANN model for our chatBot.

In [283]:
#function to return processed string for Bagofwords if  
def preprocessInputBow(inputStr):
  inputStr=bow.transform([inputStr])
  return inputStr


In [284]:
#Function to return processed string using tokeniser for ANN
def preprocessInput(inputStr):
  inputStr= tokenizer.texts_to_sequences([inputStr])
  inputStr=pad_sequences(inputStr, truncating='pre', maxlen=max_len)
  return inputStr

In [286]:
#Sample string
inp='Hello'
inp= preprocessInput(inp)
result = model_sgd.predict(inp)
print(result)
tag = ohe.inverse_transform(result)
print(tag)
for i in ChatBot_corpus['intents']:
    #print(i)
    if i['tag'] == tag:
        print( "ChatBot:" ,np.random.choice(i['responses']))

[[0.07488486 0.10495135 0.40243965 0.03086603 0.01727828 0.03178379
  0.00191605 0.00905633 0.07167643]]
[['Intro']]
ChatBot: good afternoon


In [267]:
#Defining and initiating the chatbot. The bot will accept inputs from user and provide appropriate resolution until the user inputs 'quit'
def chat():
  while True:
    print("User: " )
    inp = input()
    if inp.lower() == "quit":
        break
    inp= preprocessInput(inp)
    result = model_sgd.predict(inp)
    tag = ohe.inverse_transform(result)

    for i in ChatBot_corpus['intents']:
        if i['tag'] == tag:
            print( "ChatBot:"  , np.random.choice(i['responses']))


In [268]:
def chatWithBot():
  print("Start messaging with the bot (type quit to stop)!")
  print( "\nHi there ! I am your virtual assistant. How may I help you?")
  chat()

In [287]:
chatWithBot(How )

Start messaging with the bot (type quit to stop)!

Hi there ! I am your virtual assistant. How may I help you?
User: 
Hello
ChatBot: Hello
User: 
Hi
ChatBot: Hello
User: 
How are you
ChatBot: Hello
User: 
who are you
ChatBot: Any time!
User: 
Help me with machine learning
ChatBot: Link: Olympus wiki
User: 
I need help to understand machine learning
ChatBot: Link: Machine Learning wiki 
User: 
what is a svm
ChatBot: Link: Machine Learning wiki 
User: 
what is knn model
ChatBot: Link: Machine Learning wiki 
User: 
what is supervised learning
ChatBot: Link: Neural Nets wiki
User: 
what is neural networks
ChatBot: Link: Neural Nets wiki
User: 
what is deep learning
ChatBot: Link: Neural Nets wiki
User: 
what is meant by SGD
ChatBot: Link: Neural Nets wiki
User: 
what is meant by ANN
ChatBot: Link: Neural Nets wiki
User: 
what do u undertand by epoch
ChatBot: Link: Neural Nets wiki
User: 
explain relu
ChatBot: Link: Neural Nets wiki
User: 
explain CNN
ChatBot: Link: Neural Nets wiki
User: 


#### Conclusion/Observation:
1. There are two types of chatbots that can be designed : 
Rule based and AI based. Rule based will address and answer queries based on some pre-defined rules.
Self learning bots train and learn from the data and provide responses. 
We have created retrieval based chatbot , a subset of AI/Self learning chatbot, which provides responses based on patterns/intents entered by used. It is usually trained using large quantity of corpus.

2. We have created and compared both machine learning and NN models. Using Bag of words vactorization , we have obtained around 35-45% accuracy whereas using NN/DL , we have achieved 73.8% accuracy.
3. Although for some of the queries were not correct, chatbot was able to provide appropriate responses using DL model. 